In [1]:
import sys
# sys.path.append("../../..")
# sys.path.append("..")

In [2]:
from pathlib import Path
import numpy as np
import pandas as pd
import scipy.signal
from scipy import signal
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import RidgeCV
from sklearn.preprocessing import RobustScaler
from tqdm import tqdm
import copy

import warnings
warnings.simplefilter("ignore")

In [3]:
name = "tea"
version = 1
n_fold = 10
seed = 430

In [4]:
DATA_DIR = Path("/home/knikaido/work/Ventilator-Pressure-Prediction/data/ventilator-pressure-prediction")
OUTPUT_DIR = Path('./output/')
OOF_DIR = Path("/home/knikaido/work/Ventilator-Pressure-Prediction/data/team_oofs_stacking")
SUB_DIR = Path("/home/knikaido/work/Ventilator-Pressure-Prediction/data/team_subs_stacking")

In [5]:
sys.path.append('../../src/')
import utils as utils
from utils import Timer

In [50]:
train = pd.read_csv(DATA_DIR / "train.csv")
test = pd.read_csv(DATA_DIR / "test.csv")

In [51]:
unique_pressures = train["pressure"].unique()

In [7]:
oof_paths = sorted(list(OOF_DIR.rglob('*.npy'))+list(OOF_DIR.rglob('*.csv')))
sub_paths = sorted(list(SUB_DIR.rglob('*.npy'))+list(SUB_DIR.rglob('*.csv')))

len(oof_paths), len(sub_paths)

(15, 15)

In [8]:
oof_paths

[PosixPath('/home/knikaido/work/Ventilator-Pressure-Prediction/data/team_oofs_stacking/coorabi_lgbm_stacking_exp001_oof.npy'),
 PosixPath('/home/knikaido/work/Ventilator-Pressure-Prediction/data/team_oofs_stacking/coorabi_lgbm_stacking_exp002_oof.npy'),
 PosixPath('/home/knikaido/work/Ventilator-Pressure-Prediction/data/team_oofs_stacking/oof_currypurin_917.npy'),
 PosixPath('/home/knikaido/work/Ventilator-Pressure-Prediction/data/team_oofs_stacking/tea_stacking_v1_oof_cv0.1232.npy'),
 PosixPath('/home/knikaido/work/Ventilator-Pressure-Prediction/data/team_oofs_stacking/tea_stacking_v2_oof_cv0.1230.npy'),
 PosixPath('/home/knikaido/work/Ventilator-Pressure-Prediction/data/team_oofs_stacking/tea_stacking_v3_oof_cv0.1209.npy'),
 PosixPath('/home/knikaido/work/Ventilator-Pressure-Prediction/data/team_oofs_stacking/tea_stacking_v4_oof_cv0.1211.npy'),
 PosixPath('/home/knikaido/work/Ventilator-Pressure-Prediction/data/team_oofs_stacking/tea_stacking_v5_oof_cv0.1216.npy'),
 PosixPath('/home/

In [9]:
sub_paths

[PosixPath('/home/knikaido/work/Ventilator-Pressure-Prediction/data/team_subs_stacking/coorabi_lgbm_stacking_exp001_submission_median.csv'),
 PosixPath('/home/knikaido/work/Ventilator-Pressure-Prediction/data/team_subs_stacking/coorabi_lgbm_stacking_exp002_submission_median.csv'),
 PosixPath('/home/knikaido/work/Ventilator-Pressure-Prediction/data/team_subs_stacking/preds_currypurin_917.npy'),
 PosixPath('/home/knikaido/work/Ventilator-Pressure-Prediction/data/team_subs_stacking/tea_stacking_v1_sub_cv0.1232.csv'),
 PosixPath('/home/knikaido/work/Ventilator-Pressure-Prediction/data/team_subs_stacking/tea_stacking_v2_sub_cv0.1230.csv'),
 PosixPath('/home/knikaido/work/Ventilator-Pressure-Prediction/data/team_subs_stacking/tea_stacking_v3_sub_cv0.1209.csv'),
 PosixPath('/home/knikaido/work/Ventilator-Pressure-Prediction/data/team_subs_stacking/tea_stacking_v4_sub_cv0.1211.csv'),
 PosixPath('/home/knikaido/work/Ventilator-Pressure-Prediction/data/team_subs_stacking/tea_stacking_v5_sub_cv0.

In [10]:
def read_team_preds(paths):
    oofs = []
    for path in paths:
        path = str(path)
        if '.csv' in path:
            try:
                oof_ = pd.read_csv(path, usecols=['pressure']).values.reshape(-1)

            except:
                oof_ = pd.read_csv(path, usecols=['pred']).values.reshape(-1)
                print(type(oof_))
        else:
            try:
                oof_ = np.load(path)
            except:
                oof_ = np.load(path).reshape(-1)
#         print(f'loaded {path}')

        oofs.append(oof_)
    print('preparation done!')
    oofs = np.array(oofs)
    return oofs

In [11]:
oofs = read_team_preds(oof_paths)
_oof = pd.DataFrame(oofs.T, columns=[f'oof{str(i)}' for i in range(oofs.shape[0])])
_oof

preparation done!


,oof0,oof1,oof2,oof3,oof4,oof5,oof6,oof7,oof8,oof9,oof10,oof11,oof12,oof13,oof14
0,5.789477,5.833371,5.787222,5.756698,5.625724,5.696933,5.693756,5.820899,5.523668,5.862573,5.773038,5.810369,5.941694,5.814224,5.805068
1,5.863910,5.938485,5.887587,5.857983,5.831684,5.795729,5.917422,5.824658,5.929617,5.848502,5.877314,5.871547,5.846353,5.847313,5.846550
2,8.144781,8.245914,8.147405,7.946752,7.917322,7.891675,7.924459,8.030790,8.095352,7.955888,8.036733,8.126698,8.084612,8.069213,7.975787
3,12.079843,12.079426,12.057881,11.937506,12.020861,12.114049,12.247079,11.877783,12.134422,12.016686,11.990269,12.084424,12.239206,12.118088,12.002528
4,12.521106,12.523297,12.538396,12.329441,12.381235,12.404347,12.404102,12.507798,12.600920,12.537569,12.526273,12.546120,12.477226,12.604493,12.502975
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2290963,29.489279,29.465751,29.468914,29.485389,29.470707,29.448645,29.442779,29.476760,29.459700,29.461472,29.469219,29.480509,29.463127,29.460402,29.423048
2290964,29.105459,29.107071,29.119773,29.157852,29.136485,29.126780,29.128193,29.140826,29.138840,29.109639,29.110482,29.120060,29.111472,29.111569,29.086384
2290965,29.884453,29.874415,29.878339,29.897820,29.900912,29.885802,29.884028,29.887930,29.883578,29.880339,29.886278,29.890086,29.896856,29.892732,29.874448
2290966,29.383950,29.389308,29.391641,29.399083,29.391791,29.384078,29.388361,29.380993,29.397045,29.373224,29.380328,29.373083,29.391071,29.388413,29.382864


In [12]:
subs = read_team_preds(sub_paths)
_sub = pd.DataFrame(subs.T, columns=[f'sub{str(i)}' for i in range(oofs.shape[0])])
_sub

preparation done!


,sub0,sub1,sub2,sub3,sub4,sub5,sub6,sub7,sub8,sub9,sub10,sub11,sub12,sub13,sub14
0,6.271525,6.274139,6.234902,6.255425,6.265242,6.247533,6.264675,6.255715,6.270192,6.234668,6.242348,6.259692,6.234512,6.244083,6.239510
1,6.002286,6.000910,5.972803,5.986150,6.000848,5.981188,5.978864,5.977430,5.970970,5.970536,5.979932,5.984572,5.982571,5.995547,5.973396
2,7.146135,7.145276,7.127580,7.167441,7.192010,7.173149,7.185232,7.164241,7.154575,7.120587,7.118664,7.120526,7.149453,7.138753,7.121976
3,7.743807,7.765169,7.751527,7.754229,7.762765,7.737041,7.747494,7.752016,7.730515,7.684845,7.707492,7.684567,7.772260,7.740436,7.688272
4,9.226192,9.219263,9.211786,9.236276,9.240180,9.230789,9.238264,9.225886,9.231619,9.221489,9.240027,9.252678,9.230925,9.249313,9.231130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4023995,13.851959,13.629684,0.000000,15.082446,17.716085,12.856476,14.043241,13.823712,13.164266,14.365964,14.153107,15.354063,14.213044,14.063761,13.676671
4023996,14.090833,13.785939,0.000000,15.446876,18.200758,12.813913,13.845679,13.816230,13.322442,14.781502,14.449735,15.355318,14.399313,13.898353,14.014549
4023997,14.094390,14.146443,0.000000,15.351827,18.009788,12.774682,13.994447,13.984568,13.530339,14.877322,14.795870,15.515092,14.419702,14.630459,14.213983
4023998,14.462595,14.354524,0.000000,16.402431,19.065456,13.331053,14.225302,14.286920,13.157774,15.085758,14.273345,16.102294,14.306656,14.117593,14.409499


In [13]:
def get_raw_features(input_df, dataType = 'train'):
    colum = ['time_step', 'u_in', 'R', 'C']

    return input_df[colum]

In [14]:
def get_category_features(input_df, dataType = 'train'):
    output_df = copy.deepcopy(input_df)
    colum = ['R_C']
    rc_map = {'5_10': 0, '5_20': 1, '5_50': 2, '20_10': 3, '20_20': 4, '20_50': 5, '50_10': 6, '50_20': 7, '50_50': 8}
    
    output_df['R_C'] = [f'{r}_{c}' for r, c in zip(output_df['R'], output_df['C'])]
    output_df['R_C'] = output_df['R_C'].map(rc_map)

    return output_df[colum]

In [15]:
def get_simple_calc_features(input_df, dataType = 'train'):
    output_df = copy.deepcopy(input_df)
    c_num = input_df.shape[1]
    
    output_df['time_delta'] = output_df.groupby('breath_id')['time_step'].diff().fillna(0)
    output_df['delta'] = output_df['time_delta'] * output_df['u_in']
    output_df['area'] = output_df.groupby('breath_id')['delta'].cumsum()

    output_df['cross']= output_df['u_in']*output_df['u_out']
    output_df['cross2']= output_df['time_step']*output_df['u_out']
    
    output_df['u_in_cumsum'] = (output_df['u_in']).groupby(output_df['breath_id']).cumsum()
    output_df['one'] = 1
    output_df['count'] = (output_df['one']).groupby(output_df['breath_id']).cumsum()
    output_df['u_in_cummean'] =output_df['u_in_cumsum'] / output_df['count']
    
    output_df['u_in_sqrt'] = output_df['u_in'].apply(lambda x: np.sqrt(x))
    output_df['u_in_sqrt_cumsum'] = output_df.groupby('breath_id')['u_in_sqrt'].cumsum()
    
    output_df = output_df.drop(['count','one'], axis=1)
    
    return output_df.iloc[:, c_num:]

In [16]:
def get_agg_features(input_df, dataType = 'train'):
    
    output_df = copy.deepcopy(input_df)
    c_num = input_df.shape[1]
    
    # Dict for aggregations
    create_feature_dict = {
        'u_in': [np.max, np.mean],
    }
    
    def get_agg_window(start_time=0, end_time=3.0, add_suffix = False):
        
        df_tgt = output_df[(output_df['time_step'] >= start_time) & (output_df['time_step'] <= end_time)]
        df_feature = df_tgt.groupby(['breath_id']).agg(create_feature_dict)
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(start_time) + '_' + str(end_time))
            
        return df_feature
    
    df_agg_feature = get_agg_window().reset_index()
    
#     df_tmp = get_agg_window(start_time = 2, add_suffix = True).reset_index()
#     df_agg_feature = df_agg_feature.merge(df_tmp, how = 'left', on = 'breath_id')
#     df_tmp = get_agg_window(start_time = 1, add_suffix = True).reset_index()
#     df_agg_feature = df_agg_feature.merge(df_tmp, how = 'left', on = 'breath_id')
#     df_tmp = get_agg_window(end_time = 1, add_suffix = True).reset_index()
#     df_agg_feature = df_agg_feature.merge(df_tmp, how = 'left', on = 'breath_id')
#     df_tmp = get_agg_window(end_time = 2, add_suffix = True).reset_index()
#     df_agg_feature = df_agg_feature.merge(df_tmp, how = 'left', on = 'breath_id')

    output_df = pd.merge(output_df, df_agg_feature, how='left', on='breath_id')
    
    output_df['u_in_diffmax'] = output_df['u_in_amax'] - output_df['u_in']
    output_df['u_in_diffmean'] = output_df['u_in_mean'] - output_df['u_in']
    
#     output_df = output_df.drop(['u_in_amax','u_in_mean'], axis=1)
    
    return output_df.iloc[:, c_num:]

In [17]:
def get_half_features(input_df, dataType = 'train'):
    
    output_df = copy.deepcopy(input_df)
    c_num = input_df.shape[1]
    
    output_df['tmp'] = output_df['u_out']*(-1)+1 # inversion of u_out
    output_df['u_in_half'] = output_df['tmp'] * output_df['u_in']
    
#     u_in_half_max_dict = train.groupby('breath_id')['u_in_half'].max().to_dict()
#     train['u_in_half_max'] = train['breath_id'].map(u_in_half_max_dict)
#     u_in_half_min_dict = train.groupby('breath_id')['u_in_half'].min().to_dict()
#     train['u_in_half_min'] = train['breath_id'].map(u_in_half_min_dict)
    u_in_half_mean_dict = output_df.groupby('breath_id')['u_in_half'].mean().to_dict()
    output_df['u_in_half_mean'] = output_df['breath_id'].map(u_in_half_mean_dict)
#     u_in_half_std_dict = train.groupby('breath_id')['u_in_half'].std().to_dict()
#     train['u_in_half_std'] = train['breath_id'].map(u_in_half_std_dict)

    del output_df['u_in_half'], output_df['tmp']
    return output_df.iloc[:, c_num:]

In [18]:
def lowpass_filter(series, b, a):
    return signal.filtfilt(b, a, series)    

In [19]:
def get_filter_features(input_df, dataType = 'train'):
    
    output_df = copy.deepcopy(input_df)
    c_num = input_df.shape[1]
    
    fp = 5 # 通過域端周波数[Hz]
    fs = 10 # 阻止域端周波数[Hz]
    gpass = 3 # 通過域端最大損失[dB]
    gstop = 40 # 阻止域端最小損失[dB]
    samplerate = 100

    fn = samplerate / 2   #ナイキスト周波数
    wp = fp / fn  #ナイキスト周波数で通過域端周波数を正規化
    ws = fs / fn  #ナイキスト周波数で阻止域端周波数を正規化
    N, Wn = signal.buttord(wp, ws, gpass, gstop)  #オーダーとバターワースの正規化周波数を計算
    b, a = signal.butter(N, Wn, "low")            #フィルタ伝達関数の分子と分母を計算
    
    def get_agg_window(start_time=0, end_time=3.0, add_suffix = False):
        
        df_tgt = output_df[(output_df['time_step'] >= start_time) & (output_df['time_step'] <= end_time)]
        df_feature = df_tgt.groupby(['breath_id'])['u_in'].apply(lowpass_filter, b=b, a=a)
        df_feature.name = 'u_in_filter'
                    
        return df_feature
    
    df_agg_feature = get_agg_window().reset_index()
    df_agg_feature = df_agg_feature.explode("u_in_filter").reset_index(drop=True)
    df_agg_feature['u_in_filter'] = df_agg_feature['u_in_filter'].astype(float)
        
    df_agg_feature['u_in_filter_cumsum'] = df_agg_feature.groupby('breath_id')['u_in_filter'].cumsum()

    return df_agg_feature.iloc[:, 1:]

In [20]:
def get_vib_features(input_df, dataType = 'train'):
    
    output_df = copy.deepcopy(input_df)
    c_num = input_df.shape[1]
    
    output_df['u_out_diff'] = output_df['u_out'].diff()
    output_df['u_out_diff'].fillna(0, inplace=True)
    output_df['u_out_diff'].replace(-1, 0, inplace=True)
    uout1_df = output_df[output_df['u_out_diff']==1]
    
    first_df = output_df.loc[0::80,:]
    first_0_dict = dict(zip(first_df['id'], [0]*len(uout1_df)))

    output_df['u_in_diff'] = output_df['u_in'].diff()
    output_df['diff_sign'] = np.sign(output_df['u_in_diff'])
    output_df['sign_diff'] = output_df['diff_sign'].diff()
    output_df['tmp'] = output_df['id'].map(first_0_dict) # put 0, the 80row cycle
    output_df.iloc[0::80, output_df.columns.get_loc('sign_diff')] = output_df.iloc[0::80, output_df.columns.get_loc('tmp')]

    # Count the number of inversions, so take the absolute value and sum
    output_df['sign_diff'] = abs(output_df['sign_diff']) 
    sign_diff_dict = output_df.groupby('breath_id')['sign_diff'].sum().to_dict()
    output_df['diff_vib'] = output_df['breath_id'].map(sign_diff_dict)
    
    return output_df['sign_diff']

In [21]:
def add_time_features(out_df, input_df, dataType = 'train'):

    USE_LAG = [-2, -1, 1, 2, 3, 4]
    lag_map = {-2: 1, -1: 2, 1: 3, 2: 4, 3: 5, 4: 6}

    out_df['breath_id'] = input_df['breath_id']
    
    for lag in USE_LAG:
        out_df[f'breath_id_lag{lag_map[lag]}']=out_df['breath_id'].shift(lag).fillna(0)
        out_df[f'breath_id_lag{lag_map[lag]}same']=np.select([out_df[f'breath_id_lag{lag_map[lag]}']==out_df['breath_id']], [1], 0)

        # u_in_filter
        out_df[f'u_in_filter_lag_{lag_map[lag]}'] = out_df['u_in_filter'].shift(lag).fillna(0) * out_df[f'breath_id_lag{lag_map[lag]}same']
        out_df[f'u_in_filter_diff_{lag_map[lag]}'] = out_df['u_in_filter'] - out_df[f'u_in_filter_lag_{lag_map[lag]}']
        # u_in_sqrt
        out_df[f'u_in_sqrt_lag_{lag_map[lag]}'] = out_df['u_in_sqrt'].shift(lag).fillna(0) * out_df[f'breath_id_lag{lag_map[lag]}same']
        out_df[f'u_in_sqrt_diff_{lag_map[lag]}'] = out_df['u_in_sqrt'] - out_df[f'u_in_sqrt_lag_{lag_map[lag]}']

        # u_in 
        out_df[f'u_in_lag_{lag_map[lag]}'] = out_df['u_in'].shift(lag).fillna(0) * out_df[f'breath_id_lag{lag_map[lag]}same']
        out_df[f'u_in_diff_{lag_map[lag]}'] = out_df['u_in'] - out_df[f'u_in_lag_{lag_map[lag]}']
        # u_out
        out_df[f'u_out_lag_{lag_map[lag]}'] = out_df['u_out'].shift(lag).fillna(0) * out_df[f'breath_id_lag{lag_map[lag]}same']

        # breath_time
    out_df[f'time_step_lag_{1}'] = out_df['time_step'].shift(1).fillna(0) * out_df[f'breath_id_lag{1}same']
    out_df[f'time_step_diff_{1}'] = out_df['time_step'] - out_df[f'time_step_lag_{1}']
        
    drop_columns = ['breath_id', 'time_step_lag_1']
    drop_columns += [f'breath_id_lag{lag_map[i]}' for i in USE_LAG]
    drop_columns += [f'breath_id_lag{lag_map[i]}same' for i in USE_LAG]
    out_df = out_df.drop(drop_columns, axis=1)
    out_df = out_df.fillna(0)
    
    return out_df

In [22]:
def get_oof_features(input_df, dataType = 'train'):
    
    output_df = copy.deepcopy(input_df)
    c_num = input_df.shape[1]
    
    for i in range(len(pred_cols)):
        output_df[f"pred_{i}"] = 0.
        output_df.loc[oof["u_out"] == 0, f"pred_{i}"] = _oof[f"oof{i}"].values
    

    
    return output_df['sign_diff']

In [23]:
def to_feature(input_df, dataType = 'train'):
    """input_df を特徴量行列に変換した新しいデータフレームを返す.
    """

    processors = [
        get_raw_features,
#         get_simple_calc_features,
#         get_agg_features,
#         get_vib_features,
#         get_half_features,
#         get_category_features,
#         get_filter_features,
    ]

    out_df = pd.DataFrame()

    for func in tqdm(processors, total=len(processors)):
        with Timer(prefix='' + func.__name__ + ' '):
            _df = func(input_df, dataType)

        # 長さが等しいことをチェック (ずれている場合, func の実装がおかしい)
        assert len(_df) == len(input_df), func.__name__
        out_df = pd.concat([out_df, _df], axis=1)
#     out_df = utils.reduce_mem_usage(out_df)
#     out_df = add_time_features(out_df, input_df)
    out_df_cols = sorted(list(out_df))
    out_df = out_df[out_df_cols]
    
    return out_df

In [24]:
train_df = to_feature(train, dataType = 'train')
test_df = to_feature(test, dataType = 'test')

  0%|          | 0/1 [00:00<?, ?it/s]

get_raw_features  0.027[s]


100%|██████████| 1/1 [00:00<00:00, 28.16it/s]

get_raw_features  0.016[s]


In [25]:
pred_cols = [f"pred_{i}" for i in range(len(_oof.columns))]

In [26]:
for i in range(len(pred_cols)):
    train_df[f"pred_{i}"] = 0.
    train_df.loc[train["u_out"] == 0, f"pred_{i}"] = _oof[f"oof{i}"].values
    
for i in range(len(pred_cols)):
    test_df[f"pred_{i}"] = 0.
    test_df.loc[:, f"pred_{i}"] = _sub[f"sub{i}"].values
    
display(train_df), display(test_df)

,C,R,time_step,u_in,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7,pred_8,pred_9,pred_10,pred_11,pred_12,pred_13,pred_14
0,50,20,0.000000,0.083334,5.789477,5.833371,5.787222,5.756698,5.625724,5.696933,5.693756,5.820899,5.523668,5.862573,5.773038,5.810369,5.941694,5.814224,5.805068
1,50,20,0.033652,18.383041,5.863910,5.938485,5.887587,5.857983,5.831684,5.795729,5.917422,5.824658,5.929617,5.848502,5.877314,5.871547,5.846353,5.847313,5.846550
2,50,20,0.067514,22.509278,8.144781,8.245914,8.147405,7.946752,7.917322,7.891675,7.924459,8.030790,8.095352,7.955888,8.036733,8.126698,8.084612,8.069213,7.975787
3,50,20,0.101542,22.808822,12.079843,12.079426,12.057881,11.937506,12.020861,12.114049,12.247079,11.877783,12.134422,12.016686,11.990269,12.084424,12.239206,12.118088,12.002528
4,50,20,0.135756,25.355850,12.521106,12.523297,12.538396,12.329441,12.381235,12.404347,12.404102,12.507798,12.600920,12.537569,12.526273,12.546120,12.477226,12.604493,12.502975
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035995,10,50,2.504603,1.489714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6035996,10,50,2.537961,1.488497,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6035997,10,50,2.571408,1.558978,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6035998,10,50,2.604744,1.272663,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


,C,R,time_step,u_in,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7,pred_8,pred_9,pred_10,pred_11,pred_12,pred_13,pred_14
0,20,5,0.000000,0.000000,6.271525,6.274139,6.234902,6.255425,6.265242,6.247533,6.264675,6.255715,6.270192,6.234668,6.242348,6.259692,6.234512,6.244083,6.239510
1,20,5,0.031904,7.515046,6.002286,6.000910,5.972803,5.986150,6.000848,5.981188,5.978864,5.977430,5.970970,5.970536,5.979932,5.984572,5.982571,5.995547,5.973396
2,20,5,0.063827,14.651675,7.146135,7.145276,7.127580,7.167441,7.192010,7.173149,7.185232,7.164241,7.154575,7.120587,7.118664,7.120526,7.149453,7.138753,7.121976
3,20,5,0.095751,21.230610,7.743807,7.765169,7.751527,7.754229,7.762765,7.737041,7.747494,7.752016,7.730515,7.684845,7.707492,7.684567,7.772260,7.740436,7.688272
4,20,5,0.127644,26.320956,9.226192,9.219263,9.211786,9.236276,9.240180,9.230789,9.238264,9.225886,9.231619,9.221489,9.240027,9.252678,9.230925,9.249313,9.231130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4023995,10,20,2.530117,4.971245,13.851959,13.629684,0.000000,15.082446,17.716085,12.856476,14.043241,13.823712,13.164266,14.365964,14.153107,15.354063,14.213044,14.063761,13.676671
4023996,10,20,2.563853,4.975709,14.090833,13.785939,0.000000,15.446876,18.200758,12.813913,13.845679,13.816230,13.322442,14.781502,14.449735,15.355318,14.399313,13.898353,14.014549
4023997,10,20,2.597475,4.979468,14.094390,14.146443,0.000000,15.351827,18.009788,12.774682,13.994447,13.984568,13.530339,14.877322,14.795870,15.515092,14.419702,14.630459,14.213983
4023998,10,20,2.631134,4.982648,14.462595,14.354524,0.000000,16.402431,19.065456,13.331053,14.225302,14.286920,13.157774,15.085758,14.273345,16.102294,14.306656,14.117593,14.409499


(None, None)

In [27]:
train_df = train_df.loc[train["u_out"] == 0].reset_index(drop=True)
test_df = test_df.loc[test["u_out"] == 0].reset_index(drop=True)

In [28]:
def add_oof_features(input_df, dataType = 'train'):
      
    # v2
    input_df["pred_mean"] = np.mean(input_df[pred_cols].values, axis=1)
    input_df["pred_median"] = np.median(input_df[pred_cols].values, axis=1)

    if dataType == 'train':
        input_df['breath_id'] = train['breath_id']
    else:
        input_df['breath_id'] = test['breath_id']
    input_df["pred_std"] = input_df[pred_cols].std(axis=1)
    input_df["pred_max"] = input_df[pred_cols].values.max(axis=1)
    input_df["pred_min"] = input_df[pred_cols].values.min(axis=1)
    input_df["pred_max-min"] = input_df["pred_max"] - input_df["pred_min"]
    input_df["pred_max-median"] = input_df["pred_max"] - input_df["pred_median"]
    input_df["pred_max-mean"] = input_df["pred_max"] - input_df["pred_mean"]
    input_df["pred_median-min"] = input_df["pred_median"] - input_df["pred_min"]
    input_df["pred_mean-min"] = input_df["pred_mean"] - input_df["pred_min"]
    input_df["pred_mean-median"] = input_df["pred_mean"] - input_df["pred_median"]
    input_df["pred_kurt"] = input_df[pred_cols].kurt(axis=1)
    for col_ in pred_cols:
        input_df[f"{col_}_past_1"] = input_df.groupby("breath_id")[f"{col_}"].shift(1)
        input_df[f"{col_}_past_2"] = input_df.groupby("breath_id")[f"{col_}"].shift(2)
        input_df[f"{col_}_past_3"] = input_df.groupby("breath_id")[f"{col_}"].shift(3)
        input_df[f"{col_}_past_4"] = input_df.groupby("breath_id")[f"{col_}"].shift(4)

        input_df[f"{col_}_diff_1"] = input_df[f"{col_}"] - input_df[f"{col_}_past_1"]
        input_df[f"{col_}_diff_2"] = input_df[f"{col_}"] - input_df[f"{col_}_past_2"]
        input_df[f"{col_}_diff_3"] = input_df[f"{col_}"] - input_df[f"{col_}_past_3"]
        input_df[f"{col_}_diff_4"] = input_df[f"{col_}"] - input_df[f"{col_}_past_4"]

    input_df["u_in_past_1"] = input_df.groupby("breath_id")["u_in"].shift(1)
    input_df["u_in_past_2"] = input_df.groupby("breath_id")["u_in"].shift(2)
    input_df["u_in_past_3"] = input_df.groupby("breath_id")["u_in"].shift(3)
    input_df["u_in_past_4"] = input_df.groupby("breath_id")["u_in"].shift(4)

    input_df["u_in_diff_1"] = input_df["u_in"] - input_df["u_in_past_1"]
    input_df["u_in_diff_2"] = input_df["u_in"] - input_df["u_in_past_2"]
    input_df["u_in_diff_3"] = input_df["u_in"] - input_df["u_in_past_3"]
    input_df["u_in_diff_4"] = input_df["u_in"] - input_df["u_in_past_4"]

    input_df["u_in_cumsum"] = input_df.groupby("breath_id")["u_in"].cumsum()

    del input_df['breath_id']
    
    return input_df

In [29]:
train_df = add_oof_features(train_df)
test_df = add_oof_features(test_df)

In [30]:
# train = oof_target.copy().reset_index(drop=True)

In [31]:
train = train.loc[train["u_out"] == 0].reset_index(drop=True)

In [32]:
train["R_C"] = train["R"].astype(str) + "_" + train["C"].astype(str)
train_gby = train.groupby("breath_id")["R_C"].agg("first").reset_index()
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed).split(train_gby, train_gby["R_C"])

fold_df = pd.DataFrame()
fold_df["id"] = train["id"]
fold_df["fold"] = -1

for fold, (train_idx, valid_idx) in enumerate(cv):
    valid_breath_ids = train_gby.iloc[valid_idx]["breath_id"].values
    idxs = train[train["breath_id"].isin(valid_breath_ids)].index.to_list()
    fold_df.loc[idxs, "fold"] = fold

In [33]:
y = train["pressure"]
X = train_df

In [34]:
X

,C,R,time_step,u_in,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,...,pred_14_diff_4,u_in_past_1,u_in_past_2,u_in_past_3,u_in_past_4,u_in_diff_1,u_in_diff_2,u_in_diff_3,u_in_diff_4,u_in_cumsum
0,50,20,0.000000,0.083334,5.789477,5.833371,5.787222,5.756698,5.625724,5.696933,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.083334
1,50,20,0.033652,18.383041,5.863910,5.938485,5.887587,5.857983,5.831684,5.795729,...,NaN,0.083334,NaN,NaN,NaN,18.299707,NaN,NaN,NaN,18.466375
2,50,20,0.067514,22.509278,8.144781,8.245914,8.147405,7.946752,7.917322,7.891675,...,NaN,18.383041,0.083334,NaN,NaN,4.126236,22.425944,NaN,NaN,40.975653
3,50,20,0.101542,22.808822,12.079843,12.079426,12.057881,11.937506,12.020861,12.114049,...,NaN,22.509278,18.383041,0.083334,NaN,0.299544,4.425781,22.725488,NaN,63.784476
4,50,20,0.135756,25.355850,12.521106,12.523297,12.538396,12.329441,12.381235,12.404347,...,6.697906,22.808822,22.509278,18.383041,0.083334,2.547028,2.846573,6.972809,25.272516,89.140326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2290963,10,50,0.834147,1.869367,29.489279,29.465751,29.468914,29.485389,29.470707,29.448645,...,NaN,2.650333,2.438287,3.783043,NaN,-0.780965,-0.568920,-1.913676,NaN,10.741030
2290964,10,50,0.867574,2.154414,29.105459,29.107071,29.119773,29.157852,29.136485,29.126780,...,1.305693,1.869367,2.650333,2.438287,3.783043,0.285047,-0.495918,-0.283873,-1.628629,12.895445
2290965,10,50,0.900917,1.304434,29.884453,29.874415,29.878339,29.897820,29.900912,29.885802,...,0.842991,2.154414,1.869367,2.650333,2.438287,-0.849980,-0.564933,-1.345899,-1.133853,14.199879
2290966,10,50,0.934309,1.733830,29.383950,29.389308,29.391641,29.399083,29.391791,29.384078,...,0.618536,1.304434,2.154414,1.869367,2.650333,0.429396,-0.420585,-0.135538,-0.916503,15.933708


In [35]:
X = X.fillna(0)
test_df = test_df.fillna(0)

In [36]:
scaler = RobustScaler()
scaler.fit(X)

RobustScaler()

In [37]:
X = pd.DataFrame(scaler.transform(X.values), columns=list(X))
test_df = pd.DataFrame(scaler.transform(test_df.values), columns=list(test_df))

In [56]:
import lightgbm as lgb

models = []
oof_prediction = np.zeros(len(X))
test_preds_lst = []

for fold in range(n_fold):
    print("---------------------------------")
    print(f"Fold {fold}")
    print("---------------------------------")

    train_idx = fold_df[fold_df["fold"] != fold].index
    valid_idx = fold_df[fold_df["fold"] == fold].index

    train_X, train_y = X.iloc[train_idx], y.iloc[train_idx]
    valid_X, valid_y = X.iloc[valid_idx], y.iloc[valid_idx]
    
#     model = RidgeCV(alphas=np.logspace(-3,10, 20))
    model = RidgeCV()
    model.fit(train_X, train_y)

#     lgb_train = lgb.Dataset(train_X, train_y)
#     lgb_valid = lgb.Dataset(valid_X, valid_y, reference=lgb_train)

#     # LightGBM parameters
#     params = {
#             'boosting_type': 'gbdt',
#             'objective': 'regression_l1',
#             'metric': 'l1',
#             'seed': seed,
#             'num_boost_round': 10000,
#             'learning_rate': 0.1,
#             'lambda_l1': 0.1,
#             'lambda_l2': 0.1,
#     }


#     model = lgb.train(params, train_set=lgb_train, valid_sets=lgb_valid, early_stopping_rounds=200, verbose_eval=100)
    models.append(model)

    oof_prediction[valid_idx] = model.predict(valid_X)
    test_pred = model.predict(test_df)
    test_preds_lst.append(test_pred)
    score = np.abs(valid_y.values - oof_prediction[valid_idx]).mean()
    print(f'fold = {fold}, score = {score}')

---------------------------------
Fold 0
---------------------------------
fold = 0, score = 0.11641262578492721
---------------------------------
Fold 1
---------------------------------
fold = 1, score = 0.11971866091112936
---------------------------------
Fold 2
---------------------------------
fold = 2, score = 0.11692096768444499
---------------------------------
Fold 3
---------------------------------
fold = 3, score = 0.11859057134391522
---------------------------------
Fold 4
---------------------------------
fold = 4, score = 0.12117030197487275
---------------------------------
Fold 5
---------------------------------
fold = 5, score = 0.12278253714884753
---------------------------------
Fold 6
---------------------------------
fold = 6, score = 0.11687543648798443
---------------------------------
Fold 7
---------------------------------
fold = 7, score = 0.11676902227952306
---------------------------------
Fold 8
---------------------------------
fold = 8, score = 0.1

In [57]:
CV = np.abs(y.values - oof_prediction).mean()
CV

0.1188658685253213

In [40]:
np.save(OUTPUT_DIR / f"stacking_oof_{version}.csv", oof_prediction)

In [52]:
oof = pd.DataFrame({'pred': oof_prediction})
oof_pp = oof['pred'].map(lambda x: unique_pressures[np.abs(unique_pressures-x).argmin()])
score = np.abs(y.values - oof_pp).mean()
print(score)

0.11675919467017902


In [53]:
sub_df = pd.read_csv(DATA_DIR / "sample_submission.csv")

In [54]:
sub_df.loc[test['u_out']==0, 'pressure'] = np.stack(test_preds_lst).mean(0)
sub_df.to_csv(OUTPUT_DIR / f"stacking_submission_mean_{version}.csv", index=None)

sub_df.loc[test['u_out']==0, 'pressure'] = np.median(np.stack(test_preds_lst), axis=0)
sub_df.to_csv(OUTPUT_DIR / f"stacking_submission_median_{version}.csv", index=None)

# Post Processing: https://www.kaggle.com/snnclsr/a-dummy-approach-to-improve-your-score-postprocess
# unique_pressures = train["pressure"].unique()
sorted_pressures = np.sort(unique_pressures)
total_pressures_len = len(sorted_pressures)

def find_nearest(prediction):
    insert_idx = np.searchsorted(sorted_pressures, prediction)
    if insert_idx == total_pressures_len:
        # If the predicted value is bigger than the highest pressure in the train dataset,
        # return the max value.
        return sorted_pressures[-1]
    elif insert_idx == 0:
        # Same control but for the lower bound.
        return sorted_pressures[0]
    lower_val = sorted_pressures[insert_idx - 1]
    upper_val = sorted_pressures[insert_idx]
    return lower_val if abs(lower_val - prediction) < abs(upper_val - prediction) else upper_val

sub_df = pd.read_csv(OUTPUT_DIR / f"stacking_submission_mean_{version}.csv")
sub_df.loc[test['u_out']==0, 'pressure'] = sub_df["pressure"].apply(find_nearest)
sub_df.to_csv(OUTPUT_DIR / f"stacking_submission_mean_pp_{version}.csv", index=None)

sub_df = pd.read_csv(OUTPUT_DIR / f"stacking_submission_median_{version}.csv")
sub_df.loc[test['u_out']==0, 'pressure'] = sub_df["pressure"].apply(find_nearest)
sub_df.to_csv(OUTPUT_DIR / f"stacking_submission_median_pp_{version}.csv", index=None)